In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
housing_train_raw = pd.read_csv("../../MachineLearning Training/datasets/housing2/train.csv")
housing_test_raw = pd.read_csv("../../MachineLearning Training/datasets/housing2/test.csv")
# print(housing_train_raw.shape,housing_test_raw.shape)
full_data = housing_train_raw.append(housing_test_raw)

print(full_data.shape)
cat = ["MSZoning","Street","LotShape","LandContour","Utilities","LotConfig","LandSlope","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","Heating","HeatingQC","CentralAir","Electrical","KitchenQual","Functional","FireplaceQu","GarageType","GarageFinish","GarageQual","GarageCond","PavedDrive","SaleType","SaleCondition"]

num = ["MasVnrArea","MSSubClass","LotFrontage","LotArea","OverallQual","OverallCond","YearBuilt","YearRemodAdd","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea","BsmtFullBath","BsmtHalfBath","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","TotRmsAbvGrd","Fireplaces","GarageYrBlt","GarageCars","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","MoSold","YrSold"]

# full=cat+num
# flag=1
# for i in range(80):
#     for j in range(75):
#         if housing_test_raw.columns[i] == full[j]:
#             flag = 0
#             break
#     if flag==1:
#         print(housing_test_raw.columns[i])
#     flag=1

housing_test_raw.info()

(2919, 81)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 1

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]
# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [4]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

num_pipe=Pipeline([
    ("num" , DataFrameSelector(num)),
    ("imputer",SimpleImputer(strategy="mean"))
    
])

cat_pipe=Pipeline([
    ("cat",DataFrameSelector(cat)),
    ("imputer",MostFrequentImputer()),
    ("encoder",OneHotEncoder(sparse=False))
])

full_pipeline=FeatureUnion(transformer_list=[
    ("num",num_pipe),
    ("cat",cat_pipe)
])

X_housing = full_pipeline.fit_transform(full_data)
y_train = housing_train_raw["SalePrice"]
X_train = X_housing[0:1460,:]
X_test = X_housing[1460:2920,:]


In [5]:
def to_df(X,housing_raw):
    prediction = pd.DataFrame(X,columns=list(housing_raw.columns),index=list(housing_raw["Id"]))
    prediction.index.names=['Id']
len(cat)

39

In [10]:
import xgboost

xg_clf = xgboost.XGBRegressor()
xg_clf.fit(X_train,y_train)
y_pred_xg = xg_clf.predict(X_test)
save_pred(y_pred_xg)
#Score : 0.21729

,SalePrice
Id,
1461,133007.109375
1462,152277.109375
1463,177917.171875
1464,196181.375000
1465,184099.406250
...,...
2915,79094.718750
2916,78512.218750
2917,175129.078125


# RandomForestClassifier

In [47]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100)
forest_clf.fit(X_train,y_train)
y_pred = forest_clf.predict(X_test)
save_pred(y_pred)
#Score : 0.21729

,SalePrice
Id,
1461,125000
1462,164900
1463,192000
1464,173000
1465,180000
...,...
2915,75000
2916,88000
2917,149900


# KNeighborsClassifier

In [13]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train,y_train)
y_pred_knn = knn_clf.predict(X_test)
save_pred(y_pred_knn)

,SalePrice
Id,
1461,131500
1462,151500
1463,130500
1464,159000
1465,108480
...,...
2915,75000
2916,83000
2917,141000


# SVR

In [18]:
from sklearn.svm import SVR 

svr = SVR(kernel="linear")
svr.fit(X_train,y_train)
y_pred_svr = knn_clf.predict(X_test)
save_pred(y_pred_svr)

,SalePrice
Id,
1461,131500
1462,151500
1463,130500
1464,159000
1465,108480
...,...
2915,75000
2916,83000
2917,141000


In [8]:
def save_pred(y):
    prediction = pd.DataFrame(y,columns=["SalePrice"],index=list(housing_test_raw["Id"]))
    prediction.index.names=['Id']
    prediction.to_csv("_prediction.csv")
    return prediction

In [17]:
save_pred(y_pred)

,SalePrice
Id,
1461,129900
1462,158000
1463,192000
1464,181000
1465,189000
...,...
2915,75000
2916,88000
2917,107000


In [16]:
from sklearn.model_selection import cross_val_score
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

c:\users\marc\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.013013698630136985